In [12]:
# Import dependencies
import cv2
import numpy as np
from ultralytics import YOLO

print("Dependencies imported successfully.")


Dependencies imported successfully.


In [13]:
# Load YOLO model
model = YOLO('yolov8n.pt')  # Replace with custom weights if available
print("YOLO model loaded successfully.")


YOLO model loaded successfully.


In [14]:
# Initialize webcam
cap = cv2.VideoCapture(0)  # Use device index (0 is the default webcam)

# Check if the webcam is accessible
if not cap.isOpened():
    print("Error: Could not access the webcam.")
else:
    print("Webcam is ready.")


Webcam is ready.


In [15]:
# Function to calculate bounding box area
def calculate_area(box):
    """Calculate the area of a bounding box."""
    x1, y1, x2, y2 = box
    return (x2 - x1) * (y2 - y1)

# Function to check if an object is static
def is_static(previous_area, current_area, threshold=0.1):
    """
    Determine if an object is static based on area change.
    Args:
        previous_area (float): The bounding box area in the previous frame.
        current_area (float): The bounding box area in the current frame.
        threshold (float): The percentage of area change considered static.
    Returns:
        bool: True if the object is static, False otherwise.
    """
    if previous_area == 0:
        return False
    change = abs(current_area - previous_area) / previous_area
    return change < threshold

# Initialize variables
tracked_objects = {}  # Track bounding box areas across frames
size_increase_threshold = 1.4  # Trigger alert for 40% increase


In [16]:
# Function to calculate proximity
def calculate_proximity(prev_box, curr_box):
    """
    Calculate the proximity of an object based on bounding box size.
    Args:
        prev_box (tuple): Previous bounding box (x1, y1, x2, y2).
        curr_box (tuple): Current bounding box (x1, y1, x2, y2).
    Returns:
        float: Proximity value normalized between 0 and 1.
    """
    prev_area = (prev_box[2] - prev_box[0]) * (prev_box[3] - prev_box[1])
    curr_area = (curr_box[2] - curr_box[0]) * (curr_box[3] - curr_box[1])
    if prev_area == 0:
        return 0
    return min(curr_area / prev_area, 1.0)  # Normalize to 0-1

# Initialize variables
prev_boxes = {}
initial_box_areas = {}  # Tracks the initial size of objects
size_increase_threshold = 1.2  # Trigger alert for 20% increase in size

print("Helper functions and variables initialized.")


Helper functions and variables initialized.


In [ ]:
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not retrieve frame from webcam.")
            break

        # Run YOLO detection
        results = model(frame)

        curr_boxes = {}
        for result in results:
            for bbox, cls in zip(result.boxes.xyxy, result.boxes.cls):
                x1, y1, x2, y2 = map(int, bbox[:4])
                label = result.names[int(cls)]

                # Calculate the current bounding box area
                curr_area = calculate_area((x1, y1, x2, y2))

                # Check if this is a new object
                if label not in tracked_objects:
                    tracked_objects[label] = curr_area
                    print(f"New object detected: {label}")
                    continue

                # Check if the object is static
                previous_area = tracked_objects[label]
                if is_static(previous_area, curr_area):
                    proximity = 0  # Static objects are safe
                    color = (0, 255, 0)  # Green for static objects
                else:
                    # Check if the object is moving closer
                    size_increase = curr_area / tracked_objects[label]
                    if size_increase > size_increase_threshold:
                        proximity = min(size_increase - 1, 1.0)  # Normalize to 0 to 1
                        print(f"Warning: {label} Approaching! Proximity: {proximity:.2f}")
                        color = (0, 0, 255)  # Red for dynamic approaching objects
                    else:
                        proximity = 0  # Safe proximity
                        color = (0, 255, 0)  # Green for safe objects

                # Update the tracked object
                tracked_objects[label] = curr_area

                # Draw bounding box and proximity info
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f'{label}: {proximity:.2f}', (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Display the live webcam feed
        cv2.imshow("YOLOv8 Live Detection", frame)

        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting live feed...")
            break
except KeyboardInterrupt:
    print("Live feed interrupted.")



0: 480x640 1 person, 1 chair, 1 couch, 49.6ms
Speed: 3.0ms preprocess, 49.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
New object detected: person
New object detected: chair
New object detected: couch

0: 480x640 1 person, 1 chair, 1 bed, 1 laptop, 43.6ms
Speed: 1.0ms preprocess, 43.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
New object detected: laptop
New object detected: bed

0: 480x640 1 person, 2 chairs, 43.6ms
Speed: 2.2ms preprocess, 43.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 42.6ms
Speed: 2.0ms preprocess, 42.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 43.6ms
Speed: 1.5ms preprocess, 43.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 44.6ms
Speed: 2.0ms preprocess, 44.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chai

: 

In [1]:
# Cleanup: release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
print("Resources released successfully.")


NameError: name 'cap' is not defined